## Libraries / Setup

In [1]:
from biomart import BiomartServer
import os
import pandas as pd
import sys
import gtfparse


In [33]:
from Bio import Entrez
from difflib import SequenceMatcher
Entrez.email = "matthew.schmitz@alleninstitute.org"

def get_ncbi_assembly_stats(species_name):
    handle = Entrez.esearch(db="assembly", term=species_name)
    record = Entrez.read(handle)
    assembly_id = record['IdList'][0]
    summary_handle = Entrez.esummary(db="assembly", id=assembly_id)
    summary = Entrez.read(summary_handle)
    
    organism_from_ncbi = summary['DocumentSummarySet']['DocumentSummary'][0]['Organism']
    match_ratio = SequenceMatcher(None, organism_from_ncbi, species_name).ratio()

    # Check if the match ratio is above a certain threshold
    # You can adjust this threshold as needed
    if match_ratio < 0.4:
        print(f"Warning: Organism from NCBI ('{organism_from_ncbi}') does not closely match the input species name ('{species_name}')!")
        return None

    print(summary['DocumentSummarySet']['DocumentSummary'][0])
    return summary['DocumentSummarySet']['DocumentSummary'][0]

species_list = ['Homo sapiens', 'Mus musculus', 'Pan paniscus', 'not animal']
for species in species_list:
    print(species)
    print(f"{species}: {get_ncbi_assembly_stats(species)}")


Homo sapiens
DictElement({'RsUid': '', 'GbUid': '47542708', 'AssemblyAccession': 'GCA_963426525.1', 'LastMajorReleaseAccession': 'GCA_963426525.1', 'LatestAccession': '', 'ChainId': '963426525', 'AssemblyName': 'human feces metagenome', 'UCSCName': '', 'EnsemblName': '', 'Taxid': '9606', 'Organism': 'Homo sapiens (human)', 'SpeciesTaxid': '9606', 'SpeciesName': 'Homo sapiens', 'AssemblyType': 'haploid', 'AssemblyStatus': 'Contig', 'AssemblyStatusSort': '6', 'WGS': 'CAUKAQ01', 'GB_BioProjects': [{'BioprojectAccn': 'PRJEB65255', 'BioprojectId': '1014690'}], 'GB_Projects': [], 'RS_BioProjects': [], 'RS_Projects': [], 'BioSampleAccn': 'SAMEA114333763', 'BioSampleId': '37336560', 'Biosource': {'InfraspeciesList': [], 'Sex': '', 'Isolate': ''}, 'Coverage': '31', 'PartialGenomeRepresentation': 'false', 'Primary': '47542698', 'AssemblyDescription': '', 'ReleaseLevel': 'Major', 'ReleaseType': 'Major', 'AsmReleaseDate_GenBank': '2023/09/16 00:00', 'AsmReleaseDate_RefSeq': '1/01/01 00:00', 'SeqRe

In [38]:
import requests

def get_ensembl_assembly_stats(species_name):
    url = f"https://rest.ensembl.org/info/assembly/{species_name}?content-type=application/json"
    response = requests.get(url)
    data = response.json()

    relevant_info = {
        'AssemblyName': data.get('assembly_name', ''),
        'AssemblyAccession': data.get('assembly_accession', ''),
        'SpeciesName': species_name.replace('_', ' ').capitalize(),
        'AssemblyType': 'haploid' if len(data.get('karyotype', [])) == 1 else 'diploid',
        'TotalLength': sum([region['length'] for region in data.get('top_level_region', [])]),
        'ChromosomeCount': len(data.get('karyotype', [])),
        'ContigCount': len([region for region in data.get('top_level_region', []) if region['coord_system'] == 'contig']),
        'ScaffoldCount': len([region for region in data.get('top_level_region', []) if region['coord_system'] == 'scaffold']),
        # Note: N50 stats would typically require a more detailed breakdown of scaffold/contig lengths which isn't immediately provided here
        'AssemblyStatus': 'Chromosome' if any([region['coord_system'] == 'chromosome' for region in data.get('top_level_region', [])]) else 'Scaffold',
        'GenebuildLastUpdate': data.get('genebuild_last_geneset_update', ''),
    }

    # This is just a subset of the fields you provided from NCBI. Some fields, like 'Coverage', 'ContigN50', and 'ScaffoldN50', are not immediately available from the Ensembl API (or at least, not from this endpoint).

    return relevant_info

species_list = ['homo_sapiens', 'mus_musculus', 'pan_paniscus']
for species in species_list:
    print(species.replace("_", " "))
    print(get_ensembl_assembly_stats(species))


homo sapiens
{'AssemblyName': 'GRCh38.p14', 'AssemblyAccession': 'GCA_000001405.29', 'SpeciesName': 'Homo sapiens', 'AssemblyType': 'diploid', 'TotalLength': 3099750718, 'ChromosomeCount': 25, 'ContigCount': 0, 'ScaffoldCount': 169, 'AssemblyStatus': 'Chromosome', 'GenebuildLastUpdate': '2023-03'}
mus musculus
{'AssemblyName': 'GRCm39', 'AssemblyAccession': 'GCA_000001635.9', 'SpeciesName': 'Mus musculus', 'AssemblyType': 'diploid', 'TotalLength': 2728222451, 'ChromosomeCount': 22, 'ContigCount': 0, 'ScaffoldCount': 39, 'AssemblyStatus': 'Chromosome', 'GenebuildLastUpdate': '2023-04'}
pan paniscus
{'AssemblyName': 'panpan1.1', 'AssemblyAccession': 'GCA_000258655.2', 'SpeciesName': 'Pan paniscus', 'AssemblyType': 'diploid', 'TotalLength': 3286643896, 'ChromosomeCount': 25, 'ContigCount': 0, 'ScaffoldCount': 10249, 'AssemblyStatus': 'Chromosome', 'GenebuildLastUpdate': '2020-03'}


In [47]:
from ftplib import FTP

# Define the species of interest
species_list = ["homo_sapiens", "mus_musculus", "danio_rerio"]

ensembl_base = "ftp.ensembl.org"
ensembl_path = "/pub/current_gtf/"
local_path='/home/matthew.schmitz/Matthew/genome/'


ftp = FTP(ensembl_base)
ftp.login()

for species in species_list:
    species_dir = ensembl_path + species + "/"
    
    # Get a list of files in the species directory
    files = ftp.nlst(species_dir)
    
    # Identify the GTF file (assuming there's only one .gtf.gz per species directory)
    gtf_file = next((f for f in files if f.endswith(".gtf.gz")), None)
    
    if gtf_file:
        local_file_name = gtf_file.split('/')[-1]
        with open(os.path.join(local_path,'ensembl',species,local_file_name), 'wb') as local_file:
            ftp.retrbinary('RETR ' + gtf_file, local_file.write)

ftp.quit()


'221 Goodbye.'

In [43]:
import requests
from ftplib import FTP

# Define the species of interest
species_list = ["homo_sapiens", "mus_musculus", "danio_rerio"]
local_path='/home/matthew.schmitz/Matthew/genome/'

# Fetching from Ensembl
ensembl_base = "ftp.ensembl.org"
ensembl_path = "/pub/current_gtf/"

ftp = FTP(ensembl_base)
ftp.login()

for species in species_list:
    file_name = f"{species}/{species}.CORE.gtf.gz"
    local_file_path = os.path.join(local_path,f"{species}.CORE.gtf.gz")
    
    with open(local_file_path, 'wb') as local_file:
        ftp.retrbinary('RETR ' + ensembl_path + file_name, local_file.write)

ftp.quit()

from ftplib import FTP
local_path='/home/matthew.schmitz/Matthew/genome/'

# Define the species of interest
species_list = ["homo_sapiens", "mus_musculus", "danio_rerio"]

ensembl_base = "ftp.ensembl.org"
ensembl_path = "/pub/current_gtf/"

ftp = FTP(ensembl_base)
ftp.login()

for species in species_list:
    species_dir = ensembl_path + species + "/"
    
    # Get a list of files in the species directory
    files = ftp.nlst(species_dir)
    
    # Identify the GTF file (assuming there's only one .gtf.gz per species directory)
    gtf_file = next((f for f in files if f.endswith(".gtf.gz")), None)
    
    if gtf_file:
        local_file_name =  os.path.join(local_path,gtf_file.split('/')[-1])
        with open(local_file_name, 'wb') as local_file:
            ftp.retrbinary('RETR ' + gtf_file, local_file.write)

ftp.quit()



error_perm: 550 Failed to open file.

In [ ]:
import requests

def get_common_name(scientific_name):
    url = f"https://rest.ensembl.org/info/species?content-type=application/json"
    response = requests.get(url)
    species_list = response.json()['species']

    for species in species_list:
        if species['name'] == scientific_name:
            return species['common_name']

    return None

# Example usage
species_names = ["homo_sapiens", "mus_musculus"]
common_names = {}

for name in species_names:
    common_name = get_common_name(name)
    if common_name:
        common_names[name] = common_name
    else:
        print(f"Couldn't find common name for {name}")

print(common_names)


## Map gene symbol to ensemble ID using gtf files

For later...

In [153]:
species_to_codes = {"gorilla": "ggorilla",
            "chimp": "ptroglodytes",
            "marmoset": "cjacchus",
            "rhesus": "mmulatta",
            "human": "hsapiens"
             }

## Locations of gtf files to map symbole to ensemble ID
gtf_paths = {"gorilla": "/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/great_apes/species/inputs/Gorilla_gorilla.gorGor4.110.gtf",
            "chimp": "/allen/programs/celltypes/workgroups/rnaseqanalysis/references/chimp/ncbi/pantro/premrna/genes/genes.gtf",
            "marmoset": "/allen/programs/celltypes/workgroups/rnaseqanalysis/references/marmoset/ncbi/mcalja1.2.pat.x/genome/genes/genes.gtf",
            "rhesus": "/allen/programs/celltypes/workgroups/rnaseqanalysis/references/macaque/ncbi/mmul10/genome/genes/genes.gtf",
            "human": "/allen/programs/celltypes/workgroups/hct/SEA-AD/RNAseq/cellxgene/input/genes.gtf"}

gtfs={}
for s in gtf_paths.keys():
    print(s)
    gtfs[s]=gtfparse.parse_gtf_and_expand_attributes(gtf_paths[s])

#Just take a peek
for s in gtfs.keys():
    print(s,species_to_codes[s])
    print(gtfs[s])


## Load tables of unmapped genes

In [4]:
gene_path='/home/matthew.schmitz/nhp_unmapped_genes'
unmapped={}
for p in os.listdir(gene_path):
    if '.csv' in p:
        s=p.split('_')[0]
        unmapped[s]=pd.read_csv(os.path.join(gene_path,p))

In [196]:
for s in unmapped.keys():
    print(s)
    print(len(unmapped[s]['gene'].unique()))


human
50242
chimp
56936
marmoset
27125
rhesus
40278
gorilla
55217


# Fetch most current biomart tables

Also appears that many missing IDs are symbols mixed in, match across several columns table

In [190]:
def download_biomart_table(dataset, filename):
    #all_attributes = [x for x in dataset.attributes.keys() if 'hsapiens' in x]
    all_attributes = [
        'ensembl_gene_id',
        'external_gene_name',
        'hsapiens_homolog_associated_gene_name',
        'hsapiens_homolog_ensembl_gene',
        'hsapiens_homolog_orthology_confidence'
    ]

    response = dataset.search({
        'attributes': all_attributes
    }, header=1)  # header=1 will include the column names

    with open(filename, 'wb') as f:
        f.write('\t'.join(all_attributes).encode('ascii')+b'\n')
        for line in response.iter_lines():
            f.write(line + b'\n')

def find_matching_rows(strings, df,colnames=None):
    matches = {}
    if colnames is None:
        colnames=df.columns
    use_df=df.loc[:,colnames]
    for s in tqdm.tqdm(strings):
        mask = use_df.isin([s]).any(axis=1)
        matches[s] = df[mask]
    return matches

def get_human_orthologs(species_name, identifiers, cache_path):
    #Returns a dictionary of {original_key: dataframe of matched rows}
    server = BiomartServer("http://www.ensembl.org/biomart")
    dataset_name = f"{species_name.lower()}_gene_ensembl"
    
    # Construct cache_path based on dataset_name
    cache_name = os.path.join(cache_path,dataset_name + "_table.txt")
    
    dataset = server.datasets[dataset_name]

    # Check cache and download if necessary
    if not os.path.exists(cache_name):
        print("Cache not found. Downloading BioMart table...")
        download_biomart_table(dataset, cache_name)
        print(f"Downloaded and saved to {cache_name}")

    # Read the table into a DataFrame
    df = pd.read_csv(cache_name, sep='\t', dtype=str)
    
    # Filter using Ensembl IDs and gene symbols and add 'original_identifier' column
    #filtered_df = df[df['ensembl_gene_id'].isin(identifiers) | df['external_gene_name'].isin(identifiers)].copy()
    #filtered_df['original_identifier'] = filtered_df.apply(lambda row: row['ensembl_gene_id'] if row['ensembl_gene_id'] in identifiers else row['external_gene_name'], axis=1)
    #orthologs = {row['original_identifier']: row.drop('original_identifier').to_dict() for _, row in filtered_df.iterrows()}

    orthologs=find_matching_rows(identifiers, df,df.columns[df.columns.str.contains('gene')])        
    return orthologs


In [ ]:
mapped={}
for s in unmapped.keys():
    print(s)
    if s=='human':
        continue
    mapped[s]=get_human_orthologs(species_to_codes[s],unmapped[s]['gene'].unique(),cache_path='/home/matthew.schmitz/cache/')


In [156]:
def concatenate_dataframes(dataframes,key_name='key'):
    concatenated = pd.concat(dataframes.values(), keys=dataframes.keys(), axis=0).reset_index(level=1, drop=True).reset_index()
    concatenated = concatenated.rename(columns={'index': key_name})
    return concatenated

mapped_dfs={}
for s in mapped.keys():
    mapped_dfs[s]=concatenate_dataframes(mapped[s],'original_id')

In [188]:
for s in mapped_dfs.keys():
    mapped_dfs[s].to_csv(os.path.join('/home/matthew.schmitz/nhp_unmapped_genes',s+'_mapped.txt'),sep='\t',header=True)

In [159]:
for s in mapped.keys():
    print(unmapped[s].shape)
    print(mapped_dfs[s].shape)

(56954, 3)
(22267, 6)
(27126, 3)
(15255, 6)
(40280, 3)
(17005, 6)
(55218, 3)
(31768, 6)


In [184]:
#Cases where one chimp symbol maps to multiple chimp ensids
mapped_dfs['chimp'].loc[mapped_dfs['chimp']['original_id'].duplicated(keep=False),:]

,original_id,ensembl_gene_id,external_gene_name,hsapiens_homolog_associated_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_orthology_confidence
203,ACTL10,ENSPTRG00000044260,NaN,ACTL10,ENSG00000288649,1
204,ACTL10,ENSPTRG00000047541,NaN,ACTL10,ENSG00000288649,1
218,ACTR2,ENSPTRG00000011990,NaN,ACTR2,ENSG00000138071,1
219,ACTR2,ENSPTRG00000033914,NaN,ACTR2,ENSG00000138071,0
287,ADAP1,ENSPTRG00000041621,NaN,ADAP1,ENSG00000105963,1
...,...,...,...,...,...,...
22199,ZNF91,ENSPTRG00000043231,NaN,ZNF91,ENSG00000167232,0
22200,ENSPTRG00000019240,ENSPTRG00000019240,ZNF92,ZNF723,ENSG00000268696,0
22201,ENSPTRG00000019240,ENSPTRG00000019240,ZNF92,ZNF737,ENSG00000237440,0
22257,ZXDA,ENSPTRG00000021962,NaN,ZXDA,ENSG00000198205,1


In [185]:
#cases where multiple chimp symbols map to a single ensid
mapped_dfs['chimp'].loc[mapped_dfs['chimp']['ensembl_gene_id'].duplicated(keep=False),:].sort_values('ensembl_gene_id')

,original_id,ensembl_gene_id,external_gene_name,hsapiens_homolog_associated_gene_name,hsapiens_homolog_ensembl_gene,hsapiens_homolog_orthology_confidence
10717,NBPF12,ENSPTRG00000000232,NaN,NBPF12,ENSG00000268043,0
10715,NBPF10,ENSPTRG00000000232,NaN,NBPF10,ENSG00000271425,0
10714,NBPF1,ENSPTRG00000000232,NaN,NBPF1,ENSG00000219481,0
10718,NBPF14,ENSPTRG00000000232,NaN,NBPF14,ENSG00000270629,0
10728,NBPF9,ENSPTRG00000000232,NaN,NBPF9,ENSG00000269713,0
...,...,...,...,...,...,...
20588,TSPY2,ENSPTRG00000052575,NaN,TSPY2,ENSG00000168757,0
20582,TSPY1,ENSPTRG00000052575,NaN,TSPY1,ENSG00000258992,0
12282,PGA3,ENSPTRG00000052811,NaN,PGA3,ENSG00000229859,0
12288,PGA5,ENSPTRG00000052811,NaN,PGA5,ENSG00000256713,0


In [209]:
gtf_outs={}
for s in gtfs.keys():
    print(s,species_to_codes[s])
    if s=='human':
        continue
    intermediate=get_human_orthologs(species_to_codes[s],gtfs[s]['gene_id'].unique(),cache_path='/home/matthew.schmitz/cache/')
    gtf_outs[s]=concatenate_dataframes(intermediate,'original_id')
    gtf_outs[s].to_csv(os.path.join('/home/matthew.schmitz/nhp_unmapped_genes',s+'_mapped_whole_GTF.txt'),sep='\t',header=True)

gorilla ggorilla


100%|██████████| 30084/30084 [02:10<00:00, 230.29it/s]


chimp ptroglodytes


100%|██████████| 59133/59133 [03:56<00:00, 250.38it/s]


marmoset cjacchus


100%|██████████| 35750/35750 [03:37<00:00, 164.17it/s]


rhesus mmulatta


100%|██████████| 35219/35219 [04:16<00:00, 137.14it/s]
